In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

df

In [ ]:
display(df.describe())



In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
total
#no missing values in the data

In [ ]:
df.dtypes



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns



plt.figure(figsize = (30, 25))
sns.heatmap(df.corr(), annot = True, cmap="YlGnBu")
plt.show()

No mutlicollineairity issues here. None of the variables are highly correlated to each other.

In [ ]:
sns.distplot(df)

In [ ]:
sns.displot(data=df, x="Pregnancies", kind="hist")
sns.displot(data=df, x="Glucose", kind="hist")
sns.displot(data=df, x="SkinThickness", kind="hist")

In [ ]:
#RESCALE DATA

#X= ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Y=['Outcome']

Y = df.Outcome
x = df.drop('Outcome', axis = 1)
columns = x.columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x)
data_scaled = pd.DataFrame(X, columns = columns)

#column to maintain variable names

In [ ]:
data_scaled.head(10)

In [ ]:
from sklearn.model_selection import train_test_split

# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(data_scaled, Y, test_size=0.2)



In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) 



In [ ]:
y_test.head(10)

****Logit Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
logit_acc=('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
log_score=(logreg.score(X_test, y_test))


logit_acc

In [ ]:
# confusion matrix for logit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# confusion matrix
matrix = confusion_matrix(y_test,y_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test,y_pred,labels=[1,0])
print('Classification report : \n',matrix)

Precesion (True Positive/(True Positive + False Positive)= 
Patients who where classified as high risk for diabties and had diabities.(how many positive identifications where actually correct?) 
When the model predicts that someone has diabeties it is correct 76% of the time.(focus on case positive cases in both the negative and positive side)

Recall ((True Positive/(True Positive + False Negative)=how many positve where correct from all the actual positives? 
The model correctly defines 58% of all patient who have diabties.


increase treshold --> precision improves but recall decreases
decrease teshold--> pression decrease but recall inrecases


f1-score is avergae between precision and recall


**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=300, bootstrap = True, max_features = 'sqrt')
model.fit(X_train, y_train)
y_pred_randomforest = model.predict(X_test)
acc_random_forest=('Accuracy of Random Forest on test set: {:.2f}'.format(model.score(X_test, y_test)))
score_random_forest=(model.score(X_test, y_test))
print(acc_random_forest)


In [ ]:
# confusion matrix for random forest
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# confusion matrix
matrix = confusion_matrix(y_test,y_pred_randomforest, labels=[1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test,y_pred_randomforest,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test,y_pred_randomforest,labels=[1,0])
print('Classification report : \n',matrix)

In [ ]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(model.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head(15)


In [ ]:
importances.plot.bar()


Glucouse is the most important predictor of diabeties out come, while skin thickness is the least important.


 **Gradient Boosting**

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
X, Y = make_hastie_10_2(random_state=0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)
clf_score=('Accuracy of Gradient Boosting on test set: {:.2f}'.format(clf.score(X_test, y_test)))
clf_acc=(clf.score(X_test, y_test))
clf_score

 

**Bagging**

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
seed = 8
kfold = model_selection.KFold(n_splits = 3,
                       random_state = seed)
  
# initialize the base classifier
base_cls = DecisionTreeClassifier()
  
# no. of base classifier
num_trees = 500
  
# bagging classifier
model_s = BaggingClassifier(base_estimator = base_cls,
                          n_estimators = num_trees,
                          random_state = seed)
  
#results =('Accuracy of Bagging on test set: {:.2f}'.format(model_selection.cross_val_score(model_s, X_test, y_test, cv = kfold)))
results = model_selection.cross_val_score(model, X_test, y_test, cv = kfold)

print("accuracy :")
mean_result=(results.mean())
print(mean_result)

In [ ]:
results = pd.DataFrame({
    'Model': [ 'Random Forest',"Gradient Boosting","logit_acc", "Bagging"],
    'Score': [ score_random_forest, clf_acc, log_score,  mean_result]})
#result_df = results.sort_values(by='Score', ascending=False)

#result_df = result_df.set_index('Score')

results.head(10)